In [37]:
import pandas as pd

In [38]:
import glob

In [39]:
import numpy as np

In [40]:
def scaling(input_data, offset, adjustment_factor):
    return (
        np.log(
            (input_data - offset)
            .clip(1e-6))
        .multiply(adjustment_factor)
    )

In [41]:
def capacity_adjustment_new_plants(average_expected_revenue):
    
    revenue_lifetime_new = pd.Series(index = average_expected_revenue.filter(like="new").columns)

    lifetime_new = costs.loc[:, "lifetime", :].reindex([i.split(" ")[1] for i in revenue_lifetime_new.index]).value
    lifetime_new.index = revenue_lifetime_new.index

    for plant in revenue_lifetime_new.index:

        tech = initial_capacity_table.loc[plant, "carrier"].iloc[0]
        entry = initial_capacity_table.loc[plant, "entry"].iloc[0]

        initial_capacity_table.loc[plant]

        lifetime = lifetime_new.loc[plant]

        revenue_horizon = average_expected_revenue[plant].loc[entry: min(end_year, entry+lifetime)].sum()
        revenue_beyond = average_expected_revenue.loc[end_year,plant]*max(lifetime - (end_year - entry), 0)
        revenue_lifetime_new.loc[plant] = revenue_horizon + revenue_beyond

    invest_cost_new = costs.loc[:, "investment", :].reindex([i.split(" ")[1] for i in revenue_lifetime_new.index]).value.multiply(1e3) #converstion to EUR/MW
    invest_cost_new.index = revenue_lifetime_new.index




    fixed_om_new = (
        costs.loc[:, "FOM", :]
        .reindex([i.split(" ")[1] for i in revenue_lifetime_new.index])
        .value
        .multiply(
            costs.loc[:, "investment", :]
            .reindex([i.split(" ")[1] for i in revenue_lifetime_new.index])
            .value
            .multiply(1e3)
        )
    )

    fixed_om_new.index = revenue_lifetime_new.index

    total_fixed_new = invest_cost_new + fixed_om_new.multiply(lifetime_new)

    demand_scaling_factor = (
        demand.groupby(level=2).mean()
        .reindex(revenue_lifetime_new.index.str[:4])
    )

    demand_scaling_factor.index = revenue_lifetime_new.index


    return scaling(
        revenue_lifetime_new.div(total_fixed_new),
        offset,
        adjustment_factor    
    ).multiply(
        demand_scaling_factor
    )

In [42]:
def capacity_adjustment_existing_plants(average_expected_revenue):
    

    revenue_lifetime_existing = pd.DataFrame(
        index = average_expected_revenue.index, 
        columns = [i for i in average_expected_revenue if "new" not in i]
    )

    relative_fixed_om = costs.loc[:, "FOM", :].reindex([i.split(" ")[1] for i in revenue_lifetime_existing]).value

    relative_fixed_om.index = revenue_lifetime_existing.columns

    invest_cost = (
        costs.loc[:, "investment", :]
        .reindex([i.split(" ")[1] for i in revenue_lifetime_existing])
        .value
        .multiply(1e3) #converstion to EUR/MW
    ) 

    invest_cost.index = revenue_lifetime_existing.columns
    total_fixed_om = relative_fixed_om.multiply(invest_cost).divide(100) # O&M given in %
    

    total_fixed = pd.DataFrame(
            index = average_expected_revenue.index, 
            columns = [i for i in average_expected_revenue if "new" not in i]
        )

    for plant in revenue_lifetime_existing.columns:
        
        tech = initial_capacity_table.loc[plant, "carrier"].unique()[0]

        entry = int(initial_capacity_table.loc[plant, "entry"].unique()[0])
        exit = int(initial_capacity_table.loc[plant, "exit"].unique()[0])

        for year in range(max(start_year, entry), min(exit, end_year+1)):
            revenue_lifetime_existing.loc[year, plant] = average_expected_revenue.loc[start_year:year, plant].fillna(0).sum()
            total_fixed.loc[year, plant] = (year + 1 - entry)*total_fixed_om.loc[plant]

    revenue_lifetime_existing = revenue_lifetime_existing.astype(float)
    total_fixed = total_fixed.astype(float)

    demand_scaling_factor = (
            demand.groupby(level=2).mean()
            .reindex(revenue_lifetime_existing.columns.str[:4])
        )

    demand_scaling_factor.index = revenue_lifetime_existing.columns

    return scaling(
        revenue_lifetime_existing.divide(total_fixed),
        offset,
        adjustment_factor    
    ).multiply(
        demand_scaling_factor
    )
        

In [43]:
adjustment_factor = 0.005

In [44]:
offset = 0.1

In [45]:
demand = pd.read_hdf("resources/demand.h5")

In [46]:
revenue_files = glob.glob("results/revenues/0/*")

In [47]:
initial_capacity_table = pd.read_csv("resources/capacity_tables/0.csv", index_col=[0,1])

FileNotFoundError: [Errno 2] No such file or directory: 'resources/capacity_tables/0.csv'

In [ ]:
revenues = []
climate_years = []
target_years = []

In [ ]:
for file in revenue_files:
    revenues.append(pd.read_hdf(file))

    climate_years.append(int(file.split("cy")[1][:4]))
    target_years.append(int(file.split("ty")[1][:4]))

In [ ]:
revenues = pd.concat(revenues,axis=1).T

In [ ]:
revenues.index = pd.MultiIndex.from_arrays([climate_years, target_years])

In [ ]:
costs = pd.read_csv("../technology-data/outputs/costs_2025.csv", index_col=[0,1])

In [ ]:
eva_technologies = ['CCGT', 'OCGT', 'oil', 'biomass', 'lignite', 'coal']

In [ ]:
costs = costs.reindex(eva_technologies, level=0)

In [ ]:
average_expected_revenue = revenues.groupby(level=1).mean()

In [ ]:
average_expected_revenue = average_expected_revenue[[i for i in average_expected_revenue.columns if i.split(" ")[1] in eva_technologies]]

In [ ]:
start_year = revenues.index.levels[1][0]
end_year = revenues.index.levels[1][-1]

In [22]:
capacity_adjustment_new = capacity_adjustment_new_plants(average_expected_revenue)

In [23]:
capacity_adjustment_existing = capacity_adjustment_existing_plants(average_expected_revenue)

In [24]:
p_nom_initial = initial_capacity_table.p_nom.unstack(0)

In [25]:
p_nom_min = initial_capacity_table.p_nom_min.unstack(0)

In [26]:
p_nom_max = initial_capacity_table.p_nom_max.unstack(0)

In [27]:
next_capacities_new = pd.concat(
    [
        p_nom_initial[capacity_adjustment_new.index].add(capacity_adjustment_new).unstack(),
        p_nom_min[capacity_adjustment_new.index].unstack()
    ],
    axis=1
).max(axis=1).dropna()

In [28]:
capacity_adjustment_existing = (
    capacity_adjustment_existing
    .reindex(capacity_adjustment_existing.index[::-1])
    .cummax()
    .sort_index()
)

In [29]:
next_capacities_existing = pd.concat(
    [
        p_nom_initial[capacity_adjustment_existing.columns].add(capacity_adjustment_existing).unstack(),
        p_nom_max[capacity_adjustment_existing.columns].unstack()
    ],
    axis=1
).min(axis=1)

In [36]:
capacity_adjustment_existing.filter(like="coal")

,BG00 coal exit 2028,CZ00 coal exit 2028,CZ00 coal exit 2033,DE00 coal exit 2028,DE00 coal exit 2030,DKW1 coal exit 2030,DKW1 coal exit 2033,ES00 coal exit 2028,FI00 coal exit 2030,FI00 coal exit 2033,...,NL00 coal exit 2030,PL00 coal exit 2028,PL00 coal exit 2033,PL00 coal exit 2034,RO00 coal exit 2028,RO00 coal exit 2033,SI00 coal exit 2033,SI00 coal exit 2034,SK00 coal exit 2028,SK00 coal exit 2034
2025,85.754504,143.370393,28.667214,3034.413165,2878.446835,76.593488,59.005019,1052.230479,-822.778482,-822.778482,...,628.968052,-1456.687705,-1456.687705,-1456.687705,165.029025,201.365627,44.900263,42.624256,42.637509,89.196760
2026,84.966164,86.305318,-30.649841,2978.479741,2821.580339,63.969228,46.411804,1048.898126,-822.778482,-822.778482,...,616.360684,-1456.687705,-1456.687705,-1456.687705,163.729381,200.066993,44.474119,42.197893,41.225724,87.789216
2027,83.825347,-627.335939,-627.335939,2949.317592,2791.510021,51.325666,33.887342,1047.446210,-822.778482,-822.778482,...,610.135311,-1456.687705,-1456.687705,-1456.687705,162.736044,199.073782,44.193348,41.918257,40.160478,86.720288


In [30]:
next_capacities_existing = pd.concat(
    [
        next_capacities_existing,
        p_nom_min[capacity_adjustment_existing.columns].unstack(),
    ],
    axis=1
).max(axis=1)

In [37]:
next_capacity_table = initial_capacity_table.copy()

In [38]:
next_capacities_existing.dropna(inplace=True)

In [39]:
next_capacities_new.dropna(inplace=True)

In [40]:
next_capacity_table.loc[next_capacities_existing.index, "p_nom"] = next_capacities_existing

In [41]:
next_capacity_table.loc[next_capacities_new.index, "p_nom"] = next_capacities_new